In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%config IPCompleter.greedy=True #to enable intellisense: see it when hitting tab
from sklearn.preprocessing import LabelEncoder # integer-encoding: to transform categorical variable into integer
from sklearn.preprocessing import OneHotEncoder  # OneHot-encoding
from sklearn.compose import ColumnTransformer #needed for OneHot-encoding
from sklearn.preprocessing import LabelEncoder  #needed for OneHot-encoding
from sklearn.preprocessing import StandardScaler # to scale our data
from sklearn.decomposition import PCA #PCA

# Data preposessing:

In [2]:
music_df = pd.read_csv("C:/Users/Lisa/Desktop/Leuphana/3. Semester/ML lab/SpotifyFeatures.csv", ",") 

In [3]:
#check for NA's in Dataframe:
music_df.isnull().values.any()

False

In [4]:
#search for duplicates:
search = pd.DataFrame.duplicated(music_df)
print(search[search == True])#no duplicates

Series([], dtype: bool)


In [5]:
music_df.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


### Variables:
- popularity: ?
- key: The estimated overall key of the track. Integers map to pitches using standard Pitch Class notation (E.g.: 0 = C)
- mode: modality (major or minor) of a track (Major = 1 and Minor = 0) ? 
- time_signature:  estimated overall time signature (Taktangabe) of a track. Its a notational convention to specify how many beats are in each bar (or measure).
- title: maybe there is a way to also include strings? 
- genre: one-hot encoding
- track id: integer encoding

## Transform variables:

### 1. key:

In [6]:
music_df["key"].value_counts()

C     27583
G     26390
D     24077
C#    23201
A     22671
F     20279
B     17661
E     17390
A#    15526
F#    15222
G#    15159
D#     7566
Name: key, dtype: int64

In [7]:
## we are not using an integer encoder here because the numbers for the nodes are predetermined
# create dictionary for integer mapping:
key_mapping = {"C": 0,"G": 7,"D": 2, "C#": 1, "A": 9, "F": 5, "B": 11, "E": 4, "A#": 10, "F#": 6, "G#": 8, "D#": 3}
# map the nodes to integers:
music_df["key"] = music_df.key.map(key_mapping)
music_df.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,1,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,6,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,0,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,1,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,5,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


### 2. Genre:

In [8]:
music_df["genre"].value_counts()

Comedy              9681
Soundtrack          9646
Indie               9543
Jazz                9441
Pop                 9386
Electronic          9377
Children’s Music    9353
Folk                9299
Hip-Hop             9295
Rock                9272
Alternative         9263
Classical           9256
Rap                 9232
World               9096
Soul                9089
Blues               9023
R&B                 8992
Anime               8936
Reggaeton           8927
Ska                 8874
Reggae              8771
Dance               8701
Country             8664
Opera               8280
Movie               7806
Children's Music    5403
A Capella            119
Name: genre, dtype: int64

#### Hot-encoding of variable genre: 

First way to do it: but here you need to rename the columns (ortherwise its just numbers 1,2,....)

In [9]:
# # creating instance of one-hot-encoder
# enc = OneHotEncoder(handle_unknown='ignore')
# # passing bridge-types-cat column (label encoded values of bridge_types)
# enc_df = pd.DataFrame(enc.fit_transform(music_df[['genre']]).toarray())
# # merge with main df bridge_df on key values
# music_HE_df = music_df.join(enc_df)
# music_HE_df

Second way:

In [10]:
# creating dummy dataframe
dum_df = pd.get_dummies(music_df["genre"], columns=["genre"], prefix="genre_is")
music_df = music_df.join(dum_df)
music_df = music_df.drop(["genre"],1)


#### Integer-encoding of variable genre:

In [11]:
# Integer encoding of variable genre
# other option: hot encoding ( I think thats more the "right way to do it but than we add a column for each genre)
# https://towardsdatascience.com/categorical-encoding-techniques-93ebd18e1f24
# label_encoder = LabelEncoder()
# music_df['genre'] = label_encoder.fit_transform(music_df['genre'])
music_df.head()

,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,...,genre_is_Pop,genre_is_R&B,genre_is_Rap,genre_is_Reggae,genre_is_Reggaeton,genre_is_Rock,genre_is_Ska,genre_is_Soul,genre_is_Soundtrack,genre_is_World
0,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,1,...,0,0,0,0,0,0,0,0,0,0
1,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,6,...,0,0,0,0,0,0,0,0,0,0
2,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,0,...,0,0,0,0,0,0,0,0,0,0
3,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,1,...,0,0,0,0,0,0,0,0,0,0
4,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,5,...,0,0,0,0,0,0,0,0,0,0


## 3. Mode:

In [12]:
music_df["mode"].value_counts()

Major    151744
Minor     80981
Name: mode, dtype: int64

In [13]:
music_df["mode"] = music_df["mode"].replace({"Major": 1, "Minor":0})
music_df.rename(columns={"mode" : "Mode is Major"}, inplace=True)


In [14]:
music_df["Mode is Major"].value_counts()

1    151744
0     80981
Name: Mode is Major, dtype: int64

### 4. time_signature

I will transform the string into a fraction (float) (if thats correct? -> music expert Chan what do you think?)

In [15]:
music_df["time_signature"].value_counts()

4/4    200760
3/4     24111
5/4      5238
1/4      2608
0/4         8
Name: time_signature, dtype: int64

In [16]:
# create dictionary for integer mapping:
time_signature_mapping = {"4/4": 1,"3/4": 3/4,"5/4": 5/4, "1/4": 1/4, "0/4": 0}
# map the nodes to integers:
music_df["time_signature"] = music_df.time_signature.map(time_signature_mapping)
music_df["time_signature"].value_counts()

1.00    200760
0.75     24111
1.25      5238
0.25      2608
0.00         8
Name: time_signature, dtype: int64

## Drop all columns we are not using (string valued):

I am also removing track id because I think we dont need it right now 

In [17]:
music_df = music_df.drop(["artist_name","track_name", "track_id"],1)


In [22]:
list(music_df.columns)

['popularity',
 'acousticness',
 'danceability',
 'duration_ms',
 'energy',
 'instrumentalness',
 'key',
 'liveness',
 'loudness',
 'Mode is Major',
 'speechiness',
 'tempo',
 'time_signature',
 'valence',
 'genre_is_A Capella',
 'genre_is_Alternative',
 'genre_is_Anime',
 'genre_is_Blues',
 "genre_is_Children's Music",
 'genre_is_Classical',
 'genre_is_Comedy',
 'genre_is_Country',
 'genre_is_Dance',
 'genre_is_Electronic',
 'genre_is_Folk',
 'genre_is_Hip-Hop',
 'genre_is_Indie',
 'genre_is_Jazz',
 'genre_is_Movie',
 'genre_is_Opera',
 'genre_is_Pop',
 'genre_is_R&B',
 'genre_is_Rap',
 'genre_is_Reggae',
 'genre_is_Reggaeton',
 'genre_is_Rock',
 'genre_is_Ska',
 'genre_is_Soul',
 'genre_is_Soundtrack',
 'genre_is_World']

In [19]:
for i in range(0,len(music_df.columns)-1):
    if (music_df["genre_is_Children’s Music"][i]==1):
        music_df["genre_is_Children's Music"][i]=1
    

In [20]:
music_df = music_df.drop(["genre_is_Children’s Music"],1)
len(music_df.columns)

40

# Data Analysis

## 1. First Overview 

In [ ]:
print(music_df.info())

## 2. Plots

In [ ]:
fig = plt.figure(figsize = (7,7))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Popularity', fontsize = 15)
ax.set_ylabel('Count', fontsize = 15)
ax.set_title('Histogram', fontsize = 20)
plt.hist(music_df["popularity"], facecolor = "pink", edgecolor = "r")
ax.grid()

## 3. Correlation:

In [ ]:
pd.DataFrame.corr(music_df)

## 4. PCA

Standardize the Data:

In [ ]:
music_df_scaled = music_df.copy()
#make list of all column names
col_names = list(music_df.columns.values)

features = music_df_scaled[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
music_df_scaled[col_names] = features
music_df_scaled.head()

Transform the popularity variable into ranges:

In [ ]:

bins = [0,10, 20, 30, 45, 50, 60, 70, 80, 90, 100]
Y = pd.cut(music_df["popularity"],bins, include_lowest=True, right = True) 
# Y['popularity'] = Y['popularity'].apply(str)
# Y.popularity = Y.popularity.astype(str)

In [ ]:
# drop the target variable popularity nd ID before we apply PCA 
music_df_scaled = music_df_scaled.drop(["popularity"],1)

In [ ]:
#PCA:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(music_df_scaled)
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])
finalDf = pd.concat([principalDf, Y], axis = 1)
finalDf.popularity = finalDf.popularity.apply(str)# change buckets to string for PCA plot to work
finalDf.head()

In [ ]:
fig = plt.figure(figsize = (12,12))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = ["(-0.001, 10.0]", "(10.0, 20.0]","(20.0, 30.0] ","(30.0, 40.0]","(40.0, 50.0]","(50.0, 60.0]" ,"(60.0, 70.0] ","(70.0, 80.0]","(80.0, 90.0] ","(90.0, 100.0]"]
colors = ['r', 'g', 'b', "c", "m", "y", "gray", "brown","pink","k"]
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['popularity'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

In [ ]:
#Transform the popularity variable into ranges:
bins2 = [0,25,50,75,100]
Y2 = pd.cut(music_df["popularity"],bins2, include_lowest=True, right = True) 

In [ ]:
#PCA:
finalDf2 = pd.concat([principalDf, Y2], axis = 1)
finalDf2.popularity = finalDf2.popularity.apply(str)# change buckets to string for PCA plot to work
finalDf.head()

In [ ]:
Y2.value_counts()

In [ ]:
fig2 = plt.figure(figsize = (12,12))
ax2 = fig2.add_subplot(1,1,1) 
ax2.set_xlabel('Principal Component 1', fontsize = 15)
ax2.set_ylabel('Principal Component 2', fontsize = 15)
ax2.set_title('2 component PCA', fontsize = 20)
targets2 = ["(-0.001, 25.0]", "(25.0, 50.0] ","(50.0, 75.0]","(75.0, 100.0]"]
colors2 = ['r', 'g', 'b', "k"]
for target, color in zip(targets2,colors2):
    indicesToKeep2 = finalDf2['popularity'] == target
    ax2.scatter(finalDf2.loc[indicesToKeep2, 'principal component 1']
               , finalDf2.loc[indicesToKeep2, 'principal component 2']
               , c = color
               , s = 50)
ax2.legend(targets2)
ax2.grid()

In [ ]:
#Transform the popularity variable into ranges:
Y3 = pd.qcut(music_df["popularity"],q=4) 

In [ ]:
#PCA:
principalDf3 = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])
finalDf3 = pd.concat([principalDf3, Y3], axis = 1)
finalDf3.popularity = finalDf3.popularity.apply(str)# change buckets to string for PCA plot to work
finalDf3.head()

In [ ]:
#look at the size of the new (equally distributed) bins
Y3.value_counts()

In [ ]:
fig3 = plt.figure(figsize = (12,12))
ax3 = fig3.add_subplot(1,1,1) 
ax3.set_xlabel('Principal Component 1', fontsize = 15)
ax3.set_ylabel('Principal Component 2', fontsize = 15)
ax3.set_title('2 component PCA', fontsize = 20)
targets3 = ["(-0.001, 29.0]", "(29.0, 43.0] ","(43.0, 55.0]","(55.0, 100.0]"]
colors3 = ['r', 'g', 'b', "k"]
for target, color in zip(targets3,colors3):
    indicesToKeep3 = finalDf3['popularity'] == target
    ax3.scatter(finalDf3.loc[indicesToKeep3, 'principal component 1']
               , finalDf3.loc[indicesToKeep3, 'principal component 2']
               , c = color
               , s = 50)
ax3.legend(targets3)
ax3.grid()

no big change

# Model:

## 1. Multilayer perceptron

https://www.kaggle.com/sebastienmurphy/mlp-regressor-on-movie-dataset


In [ ]:
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.neural_network import MLPRegressor

In [ ]:
X, y = music_df.drop(["popularity"],1) , music_df["popularity"]

### Split data into training and test set:

In [ ]:
X_tr, X_t, y_tr, y_t = train_test_split(X, y, 
                                        test_size=0.3, 
                                        random_state=0)

### Transform the data: 

Options:
a) Standardize: For better performance of the algorithms and PCA standardize the data (mean = 0, variance = 1)
b) rescale the data into (-1,1)

a) Standardize:

In [ ]:
scaling = StandardScaler().fit(X_tr)
X_tr = scaling.transform(X_tr)
X_t = scaling.transform(X_t)


Scaling:

In [ ]:
# scaling = MinMaxScaler(feature_range=(-1, 1)).fit(X_tr)
# X_tr = scaling.transform(X_tr)
# X_t = scaling.transform(X_t)

Model:

Define MLP function:
- Solver = adam: for large datasets
- number of neurons in each layer: hidden_layer_sizes = (100,100) (2 layers which 100 nodes each)
  -> should be choosen as power of 2 (more efficient)
- activation function: activation='relu' (the rectified linear unit function, returns f(x) = max(0, x))
- shuffle=True: shuffle the data
- tol=1e-4: Tolerance for the optimization
- random_state=1: for reproducibility of results
- max_iter: maximum number of iterations (default=200)

In [ ]:
MLP = MLPRegressor(hidden_layer_sizes=(32,), 
                   activation='relu', 
                   solver='adam',
                   shuffle=True,
                   tol=1e-4,
                   random_state=1,
                   max_iter=1000)

Calculate test results:

cv = cross_val_score(MLP, X_tr, y_tr, cv=10)
test_score = MLP.fit(X_tr, y_tr).score(X_t, y_t)
print('CV accuracy score: %0.3f' % np.mean(cv))
print('Test accuracy score: %0.3f' % (test_score))